In [1]:
import sys
sys.path.append("/Colorization/src")  # Append path to src
import time
import numpy as np

In [2]:
from train import get_parser, train
run_cmd = './src/train.py eccv16'  # Training
run_cmd = './src/train.py eccv16 --batch-per-epoch 10 --val-batch 10'  # Testing
run_cmd = './src/train.py eccv16_pretrained --batch-per-epoch 10 --val-batch 10'  # Testing

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ed_023z2 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
parser = get_parser()
train_args = parser.parse_args(run_cmd.split()[1:])
train(train_args)


Num GPUs Available: 1

Found existing prior probs, won't regenerate it
loading annotations into memory...
Done (t=9.54s)
creating index...
index created!
loading annotations into memory...
Done (t=0.32s)
creating index...
index created!


  0%|          | 0/1849 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
 11%|█▏        | 9/79 [05:46<44:58, 38.55s/it]


Epoch 1/20: train_loss=69439800.0	val_loss=63045404.0	val_auc=0.4584


  0%|          | 0/1849 [00:00<?, ?it/s]

Checkpoint saved to /Colorization/checkpoints/eccv16_pretrained-20210311_223757/cp-001-0.4584.ckpt.pth


  0%|          | 6/1849 [00:42<3:35:41,  7.02s/it]


KeyboardInterrupt: 